In [1]:
import praw
from _keys import client_id, secret_key, username, user_agent, password, db_user, db_password, db_name, db_host, db_port
import psycopg2

In [2]:
reddit = praw.Reddit(client_id=client_id, client_secret=secret_key, username=username, password=password, user_agent=user_agent)
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
cur = conn.cursor()

In [3]:
sql = """
select all_events.subreddit
from
	(select *
	from (
	select author, title as submission, created_utm as time_created, subreddit, 'post' as submission_type
	from sus_user_posts
	where title !='This account is banned and is temporarily preserved for purposes of transparency.') as posts_aggregate
	union
	select *
	from (
	select author, comment_body as submission, created_utc as time_created, subreddit, 'comment' as submission_type
	from sus_user_comments) as comments_aggregate) as all_events
group by all_events.subreddit
order by count (*) desc
"""
# Get all sus_user_names and save to list
names = []
cur.execute(sql)
for row in cur:
    names.append(row[0])

In [4]:
# tester = names[0]
# api = "https://api.pushshift.io/reddit/search/submission"
# #4/1/2015 00:00:00
# after_time = 1427860800
# #1/1/2018 00:00:00
# final_time = 1514782800
# #response limit
# response_limit = 500

In [5]:
# sql_insert = """INSERT INTO public.norm_user_posts (author, author_fullname, author_flair_text, author_is_blocked, author_premium, can_mod_post,
#  created_utm, domain, id, is_created_from_ads_ui, is_crosspostable, is_meta, is_original_content, is_reddit_media_domain, is_robot_indexable,
#  is_self, is_video, locked, over_18, permalink, subreddit_id, title, url, post_score, subreddit, top_lvl_comments, total_comments) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [6]:
# for name in names[:10]:
#     counter = after_time
#     printcount = 0
#     post_count = 0
#     stop_time = check_if_sub_is_alive(sub, final_time)
#     while counter < stop_time and post_count < 1000:
#         try:
#             request = requests.get(f'{api}?author={tester}&after={after_time}&before={final_time}&size={response_limit}')
#             response = request.json()
#             result = response["data"]
#             counter_list = []
#             for i in result:
#                 cur.execute(insert_sql, (i["author"], i["subreddit"]))
#                 counter_list.append(i["created_utc"])
#                 post_count +=1
#             conn.commit()
#             counter = max(counter_list)
#             time.sleep(1.001)
#         except Exception as e:
#             time.sleep(1)
#             print(f'api pull error --{e}')
#             print(f'{i["subreddit"]} -- {request.status_code}')

In [7]:
for name in names:
    try:
        sub = reddit.subreddit(name)
        cur.execute("""insert into subreddit_info (subreddit_name, user_count, sub_description, over18, created_utc) values (%s, %s, %s, %s, %s)""", (name, sub.subscribers, sub.public_description, sub.over18, sub.created_utc))
        conn.commit()
    except Exception as e:
        print(e)
